In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec


In [2]:
def w(L,C):
    w = 1.0 / np.sqrt(L*C)
    return w / (2*np.pi)

def C_from_L_fr(L, fr, ToPrint=True):
    '''
    L in Henry
    fr in Hz
    return C in Farads
    '''
    C = 1.0 /( L * (2*np.pi*fr)**2 )
    if ToPrint:
        print( 'L is:', round( 1e9*L, 3 ), '[nH];  fr is:', round( 1e-9*fr, 3 ), '[GHz];  C is:', round( 1e15*C, 2 ), '[fF]')
    return C

def get_Cs(Lj, fr_q):
    L_q = L_j / 2
    C_q = C_from_L_fr( L_q, fr_q )
    C_s = C_q / 2
    return C_s

########################

In [3]:
## new good function ##
def process_HFSS_frLC(filename, datlim=[None,None], title=''):
    '''
    To extract the Capsitance from simple LC oscillator (one mode, one L)
    Takes file with shape [L, fr], fit the data and returns C=const
    use datlim to crop the data
    '''
    data = np.loadtxt(filename)
    
    L_list = 1e-9*data[:,0]  ## in H
    fr_list = data[:,1]      ##in Hz
    x_data = L_list[ datlim[0]:datlim[1] ]
    y_data = fr_list[ datlim[0]:datlim[1] ]
    
    from scipy import optimize
    def freqLC(L, L_prst, C):
        L_tot = L + np.abs(L_prst)
        w = 1.0 / np.sqrt( L_tot*C )
        return w / (2*np.pi)
    init_L_prst = 0
    init_C = 50e-15
    init_params = [init_L_prst, init_C]
    params, params_covariance = optimize.curve_fit(freqLC, x_data, y_data, p0=init_params)

    [L_prst,C] = params 
    C_str = 'C = '+ str(round(1e15*C,3))+ ' [fF]'
    L_str = 'Lprst = '+ str(round(1e9*L_prst,3))+ ' [nH]'
    title_string = title+'\n'+C_str+'\n'+L_str
    print(C_str)
    print(L_str)

    ##
    fig, ax = plt.subplots()
    ax.plot(1e9*x_data, 1e-9*y_data, 'o', label='HFSS')
    ax.plot(1e9*x_data, 1e-9*freqLC(x_data, L_prst, C), '--', label='fit=(1/2pi) * ( 1/SQRT( C*(L+Lprst) ) )')

    ax.legend()
    ax.grid()
    ax.set_title(title_string)
    ax.set_xlabel('L [nH]')
    ax.set_ylabel('Freq [GHz]')

    return C

In [11]:
def get_Cs_Ct_from_fQ_fA(fr_a, fr_q, L_a, L_j):
    def get_Cs_from_frQ_FulSys(fr_q, L_j):
        w_q = fr_q * 2*np.pi
        C_s = 1.0/( L_j*w_q**2 )
        return C_s
    C_s = get_Cs_from_frQ_FulSys( fr_q, L_j )
    w_a = fr_a * 2*np.pi
    L = (L_j*L_a) / (2*L_j+L_a) ##shouldbe_correct
#     L = L_a / (1. + (0.5*L_a/L_j) )  ##wrong
#     L = (2*L_j*L_a) / (2*L_j+L_a) ##update x2 29 Dec 2021
    C_t = 1./(L * w_a**2) - 0.5*C_s
    return [C_s, C_t]

## article example ##
print('Article Example:')
[C_s, C_t] = get_Cs_Ct_from_fQ_fA(7.78e9, 6.284e9, 5.32e-9, 5.597e-9)
print('C_s=',C_s*1e15, 'fF')
print('C_t=',C_t*1e15, 'fF')

Article Example:
('C_s=', 114.60738508130959, 'fF')
('C_t=', 174.7916418824165, 'fF')


# Remy geo. prec mesh

# Cs = 105 fF, Ct = 64 fF

### 2 May 2020

In [12]:
print 'night infinity'
print 'path', ' L_j ', 'L_a', '    Cs ', '  Ct  ', '  fQ', '   fA'

[La, Lj, fr_q, fr_a] = [40e-9, 60e-9, 2.06513e9, 3.79776e9]
[Cs, Ct] = get_Cs_Ct_from_fQ_fA(fr_a, fr_q, La, Lj)
print ' #7  ',1e9*Lj, '',1e9*La , ' ', int(1e15*Cs),'   ',int(1e15*Ct),'  ', round(1e-9*fr_q,2),' ', round(1e-9*fr_a,2)

[La, Lj, fr_q, fr_a] = [40e-9, 60e-9, 2.04008e9, 3.79676e9]
[Cs, Ct] = get_Cs_Ct_from_fQ_fA(fr_a, fr_q, La, Lj)
print ' #9  ',1e9*Lj, '',1e9*La , ' ', int(1e15*Cs),'  ',int(1e15*Ct),'  ', round(1e-9*fr_q,2),' ', round(1e-9*fr_a,2)

[La, Lj, fr_q, fr_a] = [40e-9, 60e-9, 1.99688e9, 3.79066e9]
[Cs, Ct] = get_Cs_Ct_from_fQ_fA(fr_a, fr_q, La, Lj)
print ' #10 ',1e9*Lj, '',1e9*La , ' ', int(1e15*Cs),'  ',int(1e15*Ct),'  ', round(1e-9*fr_q,2),'  ', round(1e-9*fr_a,2)


night infinity
path  L_j  L_a     Cs    Ct     fQ    fA
 #7   60.0  40.0   98     67    2.07   3.8
 #9   60.0  40.0   101    66    2.04   3.8
 #10  60.0  40.0   105    64    2.0    3.79


In [13]:
print 'night infinity'
print 'path', ' L_j ', 'L_a', '    Cs ', '  Ct  ', '  fQ', '   fA'

[La, Lj, fr_q, fr_a] = [40e-9, 60e-9, 2.06513e9, 3.79776e9]
[Cs, Ct] = get_Cs_Ct_from_fQ_fA(fr_a, fr_q, La, Lj)
print ' #7  ',1e9*Lj, '',1e9*La , ' ', int(1e15*Cs),'   ',int(1e15*Ct),'  ', round(1e-9*fr_q,2),' ', round(1e-9*fr_a,2)

[La, Lj, fr_q, fr_a] = [40e-9, 60e-9, 2.04008e9, 3.79676e9]
[Cs, Ct] = get_Cs_Ct_from_fQ_fA(fr_a, fr_q, La, Lj)
print ' #9  ',1e9*Lj, '',1e9*La , ' ', int(1e15*Cs),'  ',int(1e15*Ct),'  ', round(1e-9*fr_q,2),' ', round(1e-9*fr_a,2)

[La, Lj, fr_q, fr_a] = [40e-9, 60e-9, 1.99688e9, 3.79066e9]
[Cs, Ct] = get_Cs_Ct_from_fQ_fA(fr_a, fr_q, La, Lj)
print ' #10 ',1e9*Lj, '',1e9*La , ' ', int(1e15*Cs),'  ',int(1e15*Ct),'  ', round(1e-9*fr_q,2),'  ', round(1e-9*fr_a,2)


night infinity
path  L_j  L_a     Cs    Ct     fQ    fA
 #7   60.0  40.0   98     67    2.07   3.8
 #9   60.0  40.0   101    66    2.04   3.8
 #10  60.0  40.0   105    64    2.0    3.79


In [14]:
print 'Setup2 (2%) nothing except pads&Wires 50,4,0.5 um 4passes 0.1%'
print 'path', ' L_j ', 'L_a', '    Cs ', '  Ct  ', '  fQ', '   fA'
[La, Lj, fr_q, fr_a] = [40e-9, 60e-9, 1.67540e9, 3.63991e9]
[Cs, Ct] = get_Cs_Ct_from_fQ_fA(fr_a, fr_q, La, Lj)
print '  #4',1e9*Lj, '',1e9*La , ' ', int(1e15*Cs),'  ',int(1e15*Ct),'  ', round(1e-9*fr_q,2),' ', round(1e-9*fr_a,2)

[La, Lj, fr_q, fr_a] = [40e-9, 60e-9, 1.59545e9, 3.62418e9]
[Cs, Ct] = get_Cs_Ct_from_fQ_fA(fr_a, fr_q, La, Lj)
print '  #5',1e9*Lj, '',1e9*La , ' ', int(1e15*Cs),'  ',int(1e15*Ct),'  ', round(1e-9*fr_q,2),' ', round(1e-9*fr_a,2)

print 'Setup2 (2%) nothing except pads&Wires 50,4,0.5 um 2passes 4%'
[La, Lj, fr_q, fr_a] = [15e-9, 15e-9, 3.50774e9, 5.91113e9]
[Cs, Ct] = get_Cs_Ct_from_fQ_fA(fr_a, fr_q, La, Lj)
print ' L_j ', 'L_a', '    Cs ', '  Ct  ', '  fQ', '   fA'
print 1e9*Lj, '',1e9*La , ' ', int(1e15*Cs),'  ',int(1e15*Ct),'  ', round(1e-9*fr_q,2),' ', round(1e-9*fr_a,2)

print 'Setup2 (5%) subWires,Wires,Pads, BigRect,Between,Corners = [0.5, 4, 50, 80, 50, 10] -- only 2 passes'
[La, Lj, fr_q, fr_a] = [40e-9, 60e-9, 1.17866e9, 3.51256e9]
[Cs, Ct] = get_Cs_Ct_from_fQ_fA(fr_a, fr_q, La, Lj)
print ' L_j ', 'L_a', '    Cs ', '  Ct  ', '  fQ', '   fA'
print 1e9*Lj, '',1e9*La , ' ', int(1e15*Cs),'  ',int(1e15*Ct),'  ', round(1e-9*fr_q,2),' ', round(1e-9*fr_a,2)

print '2May_001butshit'

Setup2 (2%) nothing except pads&Wires 50,4,0.5 um 4passes 0.1%
path  L_j  L_a     Cs    Ct     fQ    fA
  #4 60.0  40.0   150    52    1.68   3.64
  #5 60.0  40.0   165    45    1.6   3.62
Setup2 (2%) nothing except pads&Wires 50,4,0.5 um 2passes 4%
 L_j  L_a     Cs    Ct     fQ    fA
15.0  15.0   137    76    3.51   5.91
Setup2 (5%) subWires,Wires,Pads, BigRect,Between,Corners = [0.5, 4, 50, 80, 50, 10] -- only 2 passes
 L_j  L_a     Cs    Ct     fQ    fA
60.0  40.0   303    -15    1.18   3.51
2May_001butshit


### 1 May

In [8]:
### Check ####
print '__Mesher 100 um, SX=0.6, SY=2.20mm  1% --- finished ---'
[La, Lj, fr_q, fr_a] = [40e-9, 60e-9, 2.02570e9, 3.7957e9]
[Cs, Ct] = get_Cs_Ct_from_fQ_fA(fr_a, fr_q, La, Lj)
print ' L_j ', 'L_a', '    Cs ', '  Ct  ', '  fQ', '   fA'
print 1e9*Lj, '',1e9*La , ' ', int(1e15*Cs),'  ',int(1e15*Ct),'  ', round(1e-9*fr_q,2),' ', round(1e-9*fr_a,2)

print '__Mesher 100 um, SX=0.6, SY=2.20mm  1% not finished (true2%)'
[La, Lj, fr_q, fr_a] = [40e-9, 60e-9, 2.07086e9, 3.80110e9]
[Cs, Ct] = get_Cs_Ct_from_fQ_fA(fr_a, fr_q, La, Lj)
print ' L_j ', 'L_a', '    Cs ', '  Ct  ', '  fQ', '   fA'
print 1e9*Lj, '',1e9*La , ' ', int(1e15*Cs),'  ',int(1e15*Ct),'  ', round(1e-9*fr_q,2),' ', round(1e-9*fr_a,2)

print '__Mesher 100 um, SX=0.6, SY=2.20mm  5%'
[La, Lj, fr_q, fr_a] = [40e-9, 60e-9, 2.47743e9, 3.88071e9]
[Cs, Ct] = get_Cs_Ct_from_fQ_fA(fr_a, fr_q, La, Lj)
print ' L_j ', 'L_a', '    Cs ', '  Ct  ', '  fQ', '   fA'
print 1e9*Lj, '',1e9*La , ' ', int(1e15*Cs),'  ',int(1e15*Ct),'  ', round(1e-9*fr_q,2),' ', round(1e-9*fr_a,2)

print '__Mesher 100 um, SX=0.4, SY=2.05mm '
[La, Lj, fr_q, fr_a] = [40e-9, 60e-9, 2.07574e9, 3.76402e9]
[Cs, Ct] = get_Cs_Ct_from_fQ_fA(fr_a, fr_q, La, Lj)
print ' L_j ', 'L_a', '    Cs ', '  Ct  ', '  fQ', '   fA'
print 1e9*Lj, '',1e9*La , ' ', int(1e15*Cs),'  ',int(1e15*Ct),'  ', round(1e-9*fr_q,2),' ', round(1e-9*fr_a,2)

print '__Mesher 10 um'
[La, Lj, fr_q, fr_a] = [40e-9, 60e-9, 2.12344e9, 3.79807e9]
[Cs, Ct] = get_Cs_Ct_from_fQ_fA(fr_a, fr_q, La, Lj)
print ' L_j ', 'L_a', '    Cs ', '  Ct  ', '  fQ', '   fA'
print 1e9*Lj, '',1e9*La , ' ', int(1e15*Cs),'  ',int(1e15*Ct),'  ', round(1e-9*fr_q,2),' ', round(1e-9*fr_a,2)

print '__Mesher 20 um'
[La, Lj, fr_q, fr_a] = [40e-9, 60e-9, 2.1234e9, 3.798e9]
[Cs, Ct] = get_Cs_Ct_from_fQ_fA(fr_a, fr_q, La, Lj)
print ' L_j ', 'L_a', '    Cs ', '  Ct  ', '  fQ', '   fA'
print 1e9*Lj, '',1e9*La , ' ', int(1e15*Cs),'  ',int(1e15*Ct),'  ', round(1e-9*fr_q,2),' ', round(1e-9*fr_a,2)

folder = 'D:\\Data\\Desktop\\HFSS_results\\__Vary_parameters__\\1_Initial_Remy_Geometry\\FullWiring\\'
filename = folder + '20200430_RemyGeo_prec_mesh.dat'
print '__Mesher 50 um'
print_C_from_file(filename)

print '\nVery big difference, with another mesh!\nBefore it was:'
folder = 'D:\\Data\\Desktop\HFSS_results\\__Vary_parameters__\\1_Initial_Remy_Geometry\\FullWiring\\'
filename = folder + '200423_200_RG_LaLj_24points_ok.dat'
print '__No additional mesh'
print_C_from_file(filename)
print('\nWhy is it so?!')


__Mesher 100 um, SX=0.6, SY=2.20mm  1% --- finished ---
 L_j  L_a     Cs    Ct     fQ    fA
60.0  40.0   102    65    2.03   3.8
__Mesher 100 um, SX=0.6, SY=2.20mm  1% not finished (true2%)
 L_j  L_a     Cs    Ct     fQ    fA
60.0  40.0   98    67    2.07   3.8
__Mesher 100 um, SX=0.6, SY=2.20mm  5%
 L_j  L_a     Cs    Ct     fQ    fA
60.0  40.0   68    77    2.48   3.88
__Mesher 100 um, SX=0.4, SY=2.05mm 
 L_j  L_a     Cs    Ct     fQ    fA
60.0  40.0   97    70    2.08   3.76
__Mesher 10 um
 L_j  L_a     Cs    Ct     fQ    fA
60.0  40.0   93    70    2.12   3.8
__Mesher 20 um
 L_j  L_a     Cs    Ct     fQ    fA
60.0  40.0   93    70    2.12   3.8
__Mesher 50 um


NameError: name 'print_C_from_file' is not defined

## Remy geometry, but Pad2 in center

In [9]:
Lj = 60e-9
La = 20e-9
fr_q = 2.45333e9
fr_a = 3.59109e9

[C_s, C_t] = get_Cs_Ct_from_fQ_fA(fr_a, fr_q, La, Lj)
# [C_s, C_t] = get_Cs_Ct_from_fQ_fA(fr_q, fr_a, La, Lj)

print ' L_j ', 'L_a', '    Cs ', '  Ct  ', '  fQ', '   fA'
print 1e9*Lj, 1e9*La,' ', round(1e15*C_s), round(1e15*C_t),' ', round(1e-9*fr_q,2),'', round(1e-9*fr_a,2)


 L_j  L_a     Cs    Ct     fQ    fA
60.0 20.0   70.0 194.0   2.45  3.59


In [10]:
'''
Variation +- 2% of each LWRG
Part_Gap	Width	Part_Ratio_Middle_o_Side	fr1	fr2	
'''
folder = 'D:\\Data\\Desktop\\HFSS_results\\__Vary_parameters__\\2_StripGeometry\\'
filename = folder + '20200428_StripGeo_Vary_LWRG=2000,sv,sv,sv.dat'

L_j = 60e-9
L_a = 40e-9

print 'Gap ',' Width','RatMoS', ' Cs ', '  Ct  ', ' fQ', '   fA'
#########################

data = np.loadtxt(filename)
Part_Gap = data[:,0]
Width = data[:,1]
RatioMoS = data[:,2]
fr_q_list = data[:,3]
fr_a_list = data[:,4]

Cs_list = np.ones_like(fr_q_list)
Ct_list = np.ones_like(fr_q_list)
for i in range(len( fr_q_list )):
    [Cs_list[i], Ct_list[i]] = get_Cs_Ct_from_fQ_fA(fr_a_list[i], fr_q_list[i], L_a, L_j)
    print Part_Gap[i],Width[i],RatioMoS[i], ' ', round(1e15*Cs_list[i]),round(1e15*Ct_list[i]),' ', round(1e-9*fr_q_list[i],2),'', round(1e-9*fr_a_list[i],2)

#########################

## reshape to 3D array 
Gaps = np.unique(Part_Gap)
Widths = np.unique(Width)
Ratios = np.unique(RatioMoS)

lg = len(Gaps)
lw = len(Widths)
lr = len(Ratios)

Cs_3D_fF = np.reshape(1e15*Cs_list, [lg,lw,lr])
Ct_3D_fF = np.reshape(1e15*Ct_list, [lg,lw,lr])

'''
Cs_3D[gap_i, width_i, rat_i]
'''

###########################################################################

max_Cs = np.max(Cs_3D_fF)
min_Cs = np.min(Cs_3D_fF)
tar_Cs = 143
range_Cs = np.max([ abs(tar_Cs-max_Cs), abs(tar_Cs-min_Cs) ])
print '\nrange_Cs',range_Cs
Cs_colormin = tar_Cs - range_Cs
Cs_colormax = tar_Cs + range_Cs

max_Ct = np.max(Ct_3D_fF)
min_Ct = np.min(Ct_3D_fF)
tar_Ct = 89
range_Ct = np.max([ abs(tar_Ct-max_Ct), abs(tar_Ct-min_Ct) ])
print 'range_Ct',range_Ct
Ct_colormin = tar_Ct - range_Ct
Ct_colormax = tar_Ct + range_Ct

#########################



####### PICTURE COLORPLOTS ######

fig1 = plt.figure(constrained_layout=True, figsize=[15,10])
spec1 = gridspec.GridSpec(ncols=4, nrows=2, figure=fig1, width_ratios=[1,1,1,.1])
ax11 = fig1.add_subplot(spec1[0, 0])
ax12 = fig1.add_subplot(spec1[0, 1])
ax13 = fig1.add_subplot(spec1[0, 2])
ax14 = fig1.add_subplot(spec1[0, 3], visible=False) #colorbar

ax11.set_title('Cs; gap='+str((Gaps[0]) ))
ax11.set_xlabel('Ratio MoS')
ax11.set_ylabel('Width [um]')
c11 = ax11.pcolor(Cs_3D_fF[0,:,:], edgecolors='k', linewidths=4, vmin=Cs_colormin, vmax=Cs_colormax)
c11.set_cmap('seismic')

ax12.set_title( 'Cs; gap='+str(Gaps[1]) )
c12 = ax12.pcolor(Cs_3D_fF[1,:,:], edgecolors='k', linewidths=4, vmin=Cs_colormin, vmax=Cs_colormax)
c12.set_cmap('seismic')

ax13.set_title( 'Cs; gap='+str(Gaps[2]) )
c13 = ax13.pcolor(Cs_3D_fF[2,:,:], edgecolors='k', linewidths=4, vmin=Cs_colormin, vmax=Cs_colormax)
c13.set_cmap('seismic')

cbar1 = fig1.colorbar(c11, ax=ax14, orientation = 'vertical', fraction=1.0)

##############

ax21 = fig1.add_subplot(spec1[1, 0])
ax22 = fig1.add_subplot(spec1[1, 1])
ax23 = fig1.add_subplot(spec1[1, 2])
ax24 = fig1.add_subplot(spec1[1, 3], visible=False) #colorbar

ax21.set_title('Ct; gap='+str((Gaps[0]) ))
c21 = ax21.pcolor(Ct_3D_fF[0,:,:], edgecolors='k', linewidths=4, vmin=Ct_colormin, vmax=Ct_colormax)
c21.set_cmap('seismic')

ax22.set_title( 'Ct; gap='+str(Gaps[1]) )
c22 = ax22.pcolor(Ct_3D_fF[1,:,:], edgecolors='k', linewidths=4, vmin=Ct_colormin, vmax=Ct_colormax)
c22.set_cmap('seismic')

ax23.set_title( 'Ct; gap='+str(Gaps[2]) )
c23 = ax23.pcolor(Ct_3D_fF[2,:,:], edgecolors='k', linewidths=4, vmin=Ct_colormin, vmax=Ct_colormax)
c23.set_cmap('seismic')

cbar2 = fig1.colorbar(c21, ax=ax24, orientation = 'vertical', fraction=1.0)

#########################



Gap   Width RatMoS  Cs    Ct    fQ    fA


IOError: [Errno 2] No such file or directory: 'D:\\Data\\Desktop\\HFSS_results\\__Vary_parameters__\\2_StripGeometry\\20200428_StripGeo_Vary_LWRG=2000,sv,sv,sv.dat'

In [11]:
### Check ####
folder = 'D:\\Data\\Desktop\\HFSS_results\\__Vary_parameters__\\2_StripGeometry\\'
filename = folder + '20200428_StripGeo_check_LWRG=2000,400,3,0.05.dat'

### GOOD TARGET PARAMETERS ###
print '\nLWRG = 2000, 400, 3 ,0.05 \n'

print_C_from_file(filename)
print('\nConfirmed!')


LWRG = 2000, 400, 3 ,0.05 



NameError: name 'print_C_from_file' is not defined